**Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi

!ls /content/drive/MyDrive/data

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/bin/bash: line 1: nvidia-smi: command not found
192-168-5-106_2025-10-08-09-00-49-AM_ISPM_RolltoRoll_GEN5_ISPM_PH3_Reference_Both_Edge_Labels
192-168-5-106_2025-10-08-09-16-10-AM_ISPM_RolltoRoll_GEN5_ISPM_PH3_Increase_Both_Edge_Labels
192-168-5-106_2025-10-08-10-55-35-AM_ISPM_RolltoRoll_GEN5_ISPM_PH3_Decrease_Both_Edge_Labels
2025-05-26-03-00-51-PM_Gradually_Lowering_Pressure_0dot4mBar_Steps
2025-05-27-03-22-58-PM_Gradually_Increasing_Pressure_0dot4mBar_Steps_ISPM_PH4
fit
preprocessed
ringdown


**Extracts the third phase and returns the same tensor with less samples...**

In [ ]:
import numpy as np
import json, os

# ---------------- CONFIGURATION ----------------
replicate = 2
version = "increasing"
base_dir = "/content/drive/MyDrive/data/preprocessed"
output_dir = "/content/drive/MyDrive/data/ringdown"

# Input files
tensor_path = os.path.join(base_dir, f"tensor_replicate_{replicate}_{version}.npy")
press_path  = os.path.join(base_dir, f"pressures_replicate_{replicate}_{version}.json")
nozz_path   = os.path.join(base_dir, f"nozzles_replicate_{replicate}_{version}.json")

# Output file
os.makedirs(output_dir, exist_ok=True)
out_tensor_path = os.path.join(output_dir, f"tensor_replicate_{replicate}_{version}_ringdown.npy")
# ------------------------------------------------

# ---------------- LOAD ----------------
if not all(os.path.exists(p) for p in [tensor_path, press_path, nozz_path]):
    raise FileNotFoundError(" Missing one or more preprocessed files. Run the earlier preprocessing step first.")

tensor = np.array(np.load(tensor_path, allow_pickle=True))
with open(press_path, "r") as f:
    pressures = np.array(json.load(f), dtype=float)
with open(nozz_path, "r") as f:
    nozzles = json.load(f)

n_nozzles, n_samples, n_pressures = tensor.shape
print(f" Loaded tensor: {tensor.shape} (nozzles × samples × pressures)")

# ---------------- FIXED CROP ----------------
start_offset = 35  # remove first 35 samples
if start_offset >= n_samples:
    raise ValueError(" Start offset is larger than waveform length!")

# Compute new length
min_len = n_samples - start_offset
print(f" Cropping first {start_offset} samples → keeping {min_len} samples per waveform")

# Slice the tensor
trimmed_tensor = tensor[:, start_offset:, :]

# ---------------- SAVE ----------------
np.save(out_tensor_path, trimmed_tensor)
print(f" Saved ringdown-only tensor: {out_tensor_path}")
print(f"Shape: {trimmed_tensor.shape} (nozzles × ringdown samples × pressures)")



 Loaded tensor: (1280, 124, 16) (nozzles × samples × pressures)
 Cropping first 35 samples → keeping 89 samples per waveform
 Saved ringdown-only tensor: /content/drive/MyDrive/data/ringdown/tensor_replicate_2_increasing_ringdown.npy
Shape: (1280, 89, 16) (nozzles × ringdown samples × pressures)


**Visualization for ringdown tensor**

In [ ]:
# --- Colab + widget + plotly fixes (run once per runtime) ---
!pip -q install ipywidgets plotly

from google.colab import output
output.enable_custom_widget_manager()

import plotly.io as pio
pio.renderers.default = "colab"

# --- Visualization: Ringdown-only waveform evolution across pressures ---
import os, json
import numpy as np
import plotly.graph_objects as go
from ipywidgets import FloatRangeSlider, interact

# ---------------- CONFIGURATION ----------------
replicate = 1
version = "lowering"
target_nozzle = "B36"

# Paths
base_dir_pre  = "/content/drive/MyDrive/data/preprocessed"
base_dir_ring = "/content/drive/MyDrive/data/ringdown"

tensor_path = os.path.join(base_dir_ring, f"tensor_replicate_{replicate}_{version}_ringdown.npy")
press_path  = os.path.join(base_dir_pre,  f"pressures_replicate_{replicate}_{version}.json")
nozz_path   = os.path.join(base_dir_pre,  f"nozzles_replicate_{replicate}_{version}.json")

# ---------------- LOAD ----------------
if not all(os.path.exists(p) for p in [tensor_path, press_path, nozz_path]):
    print(f"Missing ringdown or metadata files for replicate {replicate}. Please check the paths.")
else:
    tensor = np.load(tensor_path)  # (nozzles, samples, pressures)
    with open(press_path, "r") as f:
        all_pressures = np.array(json.load(f), dtype=float)
    with open(nozz_path, "r") as f:
        nozzles = json.load(f)

    print(f"Loaded ringdown tensor: {tensor.shape} (nozzles × samples × pressures)")
    print(f"{len(nozzles)} nozzles, {len(all_pressures)} pressures")

    if target_nozzle not in nozzles:
        print(f"Nozzle {target_nozzle} not found.")
    else:
        idx = nozzles.index(target_nozzle)

        slider = FloatRangeSlider(
            value=[float(all_pressures.min()), float(all_pressures.max())],
            min=float(all_pressures.min()),
            max=float(all_pressures.max()),
            step=0.4,
            description="Pressure range:",
            continuous_update=False,
            layout={"width": "85%"}
        )

        def plot_pressure_range(pressure_range, max_traces=25):
            pmin, pmax = pressure_range
            mask = (all_pressures >= pmin) & (all_pressures <= pmax)
            sel_idx = np.where(mask)[0]
            sel_p   = all_pressures[mask]

            if len(sel_idx) == 0:
                print("No pressures in range.")
                return

            # cap number of traces for reliable rendering
            if len(sel_idx) > max_traces:
                keep = np.linspace(0, len(sel_idx)-1, max_traces, dtype=int)
                sel_idx = sel_idx[keep]
                sel_p   = sel_p[keep]

            x = np.arange(tensor.shape[1])
            fig = go.Figure()

            for i, p in zip(sel_idx, sel_p):
                fig.add_trace(go.Scatter(
                    x=x,
                    y=tensor[idx, :, i],
                    mode="lines",
                    name=f"{p:.1f} mBar",
                    hovertemplate="Pressure: %{text} mBar<br>Sample: %{x}<br>Amplitude: %{y}<extra></extra>",
                    text=[f"{p:.1f}"] * len(x),
                ))

            fig.update_layout(
                title=f"Ringdown waveforms — Nozzle {target_nozzle} ({replicate}_{version})",
                xaxis_title="Sample index (ringdown only)",
                yaxis_title="Amplitude",
                height=520,
                margin=dict(l=40, r=220, t=60, b=40),
                legend=dict(font=dict(size=9))
            )

            fig.show(renderer="colab")

        interact(plot_pressure_range, pressure_range=slider);


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.5 MB/s eta 0:00:00
Loaded ringdown tensor: (1280, 89, 95) (nozzles × samples × pressures)
1280 nozzles, 95 pressures


interactive(children=(FloatRangeSlider(value=(-87.6, -50.0), continuous_update=False, description='Pressure ra…